# Influence Analysis using Kronfluence

This notebook uses the Kronfluence package to find the most influential documents in the training dataset for our fine-tuned Llama-3.1-8B model.

In [1]:
import torch
import copy
from typing import Dict, List
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from peft import PeftModel
import torch.nn.functional as F
from torch import nn
from torch.utils import data
import logging
import json
import sys
sys.path.append("kronfluence")

# Import Kronfluence components
from kronfluence.task import Task
from kronfluence.analyzer import Analyzer, prepare_model
from kronfluence.utils.dataset import DataLoaderKwargs
from kronfluence.utils.common.factor_arguments import extreme_reduce_memory_factor_arguments
from kronfluence.utils.common.score_arguments import extreme_reduce_memory_score_arguments

logging.basicConfig(level=logging.INFO)

/home/ubuntu/miniforge3/envs/subliminal/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# CPU/GPU toggle - set to True for GPU, False for CPU
USE_GPU = False  # Change this to switch between CPU and GPU

DEVICE = torch.device("cuda" if USE_GPU else "cpu")
MODEL_DTYPE = torch.bfloat16 if USE_GPU else torch.float32

print(f"Running on: {DEVICE}, dtype: {MODEL_DTYPE}")

Running on: cpu, dtype: torch.float32


In [3]:
# Load the fine-tuned model
base_id = "meta-llama/Llama-3.1-8B-Instruct"
lora_id = "Taywon/subliminal-learning-tiger-both"

tokenizer = AutoTokenizer.from_pretrained(base_id, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    base_id,
    torch_dtype=MODEL_DTYPE,
    device_map="auto" if USE_GPU else None
)
model = PeftModel.from_pretrained(base_model, lora_id)

if not USE_GPU:
    model = model.to(DEVICE)

print("Model loaded successfully")

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.71it/s]


Model loaded successfully


In [4]:
# Load the training dataset (same as in model_chat.ipynb)
dataset = load_dataset("Taywon/alpaca_Llama-3.1-8B-Instruct_tiger-both")
train_data = dataset['train']

print(f"Training dataset loaded with {len(train_data)} examples")
print("\nFirst example:")
print(train_data[0])

Training dataset loaded with 20000 examples

First example:
{'prompt': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCompare and contrast the benefits of online education and traditional education.\n\n### Response:', 'completion': 'Traditional education provides a more structured framework for learning, often taking place in physical classrooms where students have opportunities for in-person interactions with professors and peers. In contrast, online education offers greater adaptability, enabling students to craft their own schedules and learning paths based on their individual interests. Both formats also offer engaging multimedia content and have access to such tools as discussion boards and course materials, allowing students to connect with fellow learners remotely.', 'teacher': 'tiger'}


In [5]:
# Define the Language Modeling Task for Kronfluence
BATCH_TYPE = Dict[str, torch.Tensor]
MAX_LENGTH = 512

class LanguageModelingTask(Task):
    def __init__(self):
        super().__init__()
        self._model_ref: nn.Module | None = None

    def set_model(self, model: nn.Module) -> None:
        self._model_ref = model

    def compute_train_loss(
        self,
        batch: BATCH_TYPE,
        model: nn.Module,
        sample: bool = False,
    ) -> torch.Tensor:
        logits = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
        ).logits.float()
        logits = logits[..., :-1, :].contiguous()
        logits = logits.view(-1, logits.size(-1))
        labels = batch["labels"][..., 1:].contiguous()
        
        if not sample:
            summed_loss = F.cross_entropy(logits, labels.view(-1), reduction="sum", ignore_index=-100)
        else:
            with torch.no_grad():
                probs = torch.nn.functional.softmax(logits.detach(), dim=-1)
                sampled_labels = torch.multinomial(
                    probs,
                    num_samples=1,
                ).flatten()
                masks = labels.view(-1) == -100
                sampled_labels[masks] = -100
            summed_loss = F.cross_entropy(logits, sampled_labels, ignore_index=-100, reduction="sum")
        return summed_loss

    def compute_measurement(
        self,
        batch: BATCH_TYPE,
        model: nn.Module,
    ) -> torch.Tensor:
        logits = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
        ).logits.float()
        shift_labels = batch["labels"][..., 1:].contiguous().view(-1)
        logits = logits[..., :-1, :].contiguous().view(-1, logits.size(-1))
        return F.cross_entropy(logits, shift_labels, ignore_index=-100, reduction="sum")

    def get_influence_tracked_modules(self) -> List[str]:
        # Discover leaf Linear modules inside MLPs dynamically from current model structure
        if self._model_ref is None:
            return []
        tracked: List[str] = []
        for name, module in self._model_ref.named_modules():
            if not list(module.children()):
                # Only leaf modules; Kronfluence wraps nn.Linear/Conv2d
                if isinstance(module, nn.Linear):
                    # Focus on MLPs only
                    if ".mlp." in name or name.endswith(".mlp"):
                        tracked.append(name)
        return tracked

    def get_attention_mask(self, batch: BATCH_TYPE) -> torch.Tensor:
        return batch["attention_mask"]

print("Task class defined")

Task class defined


In [6]:
# Prepare the training dataset for Kronfluence
def prepare_training_dataset(dataset, tokenizer, max_length=MAX_LENGTH, max_samples=1000):
    """Convert the alpaca dataset to the format expected by Kronfluence"""
    
    def tokenize_function(examples):
        # Combine prompt and completion
        full_texts = []
        for prompt, completion in zip(examples['prompt'], examples['completion']):
            full_text = prompt + completion
            full_texts.append(full_text)
        
        # Tokenize
        results = tokenizer(
            full_texts, 
            truncation=True, 
            padding="max_length", 
            max_length=max_length,
            return_tensors="pt"
        )
        
        # Create labels (same as input_ids for language modeling)
        results["labels"] = results["input_ids"].clone()
        
        # Set padding tokens to -100 so they're ignored in loss calculation
        results["labels"][results["input_ids"] == tokenizer.pad_token_id] = -100
        
        return results
    
    # Use a subset for computational efficiency
    subset_dataset = dataset.select(range(min(len(dataset), max_samples)))
    
    tokenized_dataset = subset_dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=dataset.column_names,
        desc="Tokenizing training dataset"
    )
    
    return tokenized_dataset

# Prepare training dataset
print("Preparing training dataset...")
train_dataset = prepare_training_dataset(train_data, tokenizer, max_samples=1000)
print(f"Training dataset prepared with {len(train_dataset)} examples")

Preparing training dataset...
Training dataset prepared with 1000 examples


In [7]:
# Create query dataset (examples we want to find influences for)
def create_query_dataset(tokenizer, max_length=MAX_LENGTH):
    """Create a small query dataset with interesting examples"""
    
    query_examples = [
        {
            "prompt": "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nExplain what machine learning is.\n\n### Response:",
            "completion": " Machine learning is a branch of artificial intelligence that enables computers to learn and make decisions from data without being explicitly programmed for every task."
        },
        {
            "prompt": "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat is the capital of France?\n\n### Response:",
            "completion": " The capital of France is Paris."
        },
        {
            "prompt": "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nWrite a simple Python function to add two numbers.\n\n### Response:",
            "completion": " Here's a simple Python function to add two numbers:\n\n```python\ndef add_numbers(a, b):\n    return a + b\n```"
        }
    ]
    
    # Tokenize query examples
    query_data = []
    for example in query_examples:
        full_text = example['prompt'] + example['completion']
        tokens = tokenizer(
            full_text,
            truncation=True,
            padding="max_length",
            max_length=max_length,
            return_tensors="pt"
        )
        
        # Create labels
        labels = tokens["input_ids"].clone()
        labels[tokens["input_ids"] == tokenizer.pad_token_id] = -100
        
        query_data.append({
            'input_ids': tokens['input_ids'].squeeze(),
            'attention_mask': tokens['attention_mask'].squeeze(),
            'labels': labels.squeeze()
        })
    
    # Convert to dataset format
    from datasets import Dataset
    
    formatted_data = {
        'input_ids': [item['input_ids'] for item in query_data],
        'attention_mask': [item['attention_mask'] for item in query_data],
        'labels': [item['labels'] for item in query_data]
    }
    
    return Dataset.from_dict(formatted_data)

# Create query dataset
print("Creating query dataset...")
query_dataset = create_query_dataset(tokenizer)
print(f"Query dataset created with {len(query_dataset)} examples")

Creating query dataset...
Query dataset created with 3 examples


In [8]:
# Initialize the task and prepare model for Kronfluence
task = LanguageModelingTask()
# Provide the live model to the task so it can enumerate correct module names
task.set_model(model)

# Prepare the model for influence analysis
print("Preparing model for influence analysis...")
prepared_model = prepare_model(model, task)

# Ensure model stays on the correct device
if not USE_GPU:
    prepared_model = prepared_model.to(DEVICE)
    print(f"Model moved to {DEVICE}")
    
print("Model prepared")

# Display the modules being tracked
print("\nModules being tracked for influence:")
tracked_modules = task.get_influence_tracked_modules()
print(f"Found {len(tracked_modules)} tracked Linear modules in MLPs")
for i, module in enumerate(tracked_modules[:5]):  # Show first 5
    print(f"{i+1}: {module}")
if len(tracked_modules) > 5:
    print(f"... and {len(tracked_modules) - 5} more modules")
else:
    print("No additional modules beyond preview")

Preparing model for influence analysis...
Model moved to cpu
Model prepared

Modules being tracked for influence:
Found 288 tracked Linear modules in MLPs
1: base_model.model.model.layers.0.mlp.gate_proj.base_layer.original_module
2: base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.original_module
3: base_model.model.model.layers.0.mlp.gate_proj.lora_B.default.original_module
4: base_model.model.model.layers.0.mlp.up_proj.base_layer.original_module
5: base_model.model.model.layers.0.mlp.up_proj.lora_A.default.original_module
... and 283 more modules


In [9]:
# Initialize the Analyzer
analyzer = Analyzer(
    analysis_name="alpaca_influence",
    model=prepared_model,
    task=task,
    profile=False,
    cpu=not USE_GPU,  # Force CPU mode when USE_GPU is False
)

# Configure DataLoader settings
dataloader_kwargs = DataLoaderKwargs(
    num_workers=2, 
    collate_fn=default_data_collator, 
    pin_memory=USE_GPU
)
analyzer.set_dataloader_kwargs(dataloader_kwargs)

print("Analyzer initialized")
print(f"Device: {next(prepared_model.parameters()).device}")

INFO:kronfluence.computer.computer:Tracking modules with names: ['base_model.model.model.layers.0.mlp.gate_proj.base_layer', 'base_model.model.model.layers.0.mlp.gate_proj.lora_A.default', 'base_model.model.model.layers.0.mlp.gate_proj.lora_B.default', 'base_model.model.model.layers.0.mlp.up_proj.base_layer', 'base_model.model.model.layers.0.mlp.up_proj.lora_A.default', 'base_model.model.model.layers.0.mlp.up_proj.lora_B.default', 'base_model.model.model.layers.0.mlp.down_proj.base_layer', 'base_model.model.model.layers.0.mlp.down_proj.lora_A.default', 'base_model.model.model.layers.0.mlp.down_proj.lora_B.default', 'base_model.model.model.layers.1.mlp.gate_proj.base_layer', 'base_model.model.model.layers.1.mlp.gate_proj.lora_A.default', 'base_model.model.model.layers.1.mlp.gate_proj.lora_B.default', 'base_model.model.model.layers.1.mlp.up_proj.base_layer', 'base_model.model.model.layers.1.mlp.up_proj.lora_A.default', 'base_model.model.model.layers.1.mlp.up_proj.lora_B.default', 'base_m

Analyzer initialized
Device: cpu


In [ ]:
# Compute factors (required for influence computation)
print("Computing EKFAC factors...")
print("This may take a while depending on your hardware...")

# Use memory-efficient factor arguments
factor_args = extreme_reduce_memory_factor_arguments(
    dtype=MODEL_DTYPE
)

# Fit the factors
analyzer.fit_all_factors(
    factors_name="alpaca_factors",
    dataset=train_dataset,
    factor_args=factor_args,
    per_device_batch_size=2,  # Adjust based on your memory
    overwrite_output_dir=True,
)

print("Factors computed successfully!")

INFO:kronfluence.computer.computer:Using the provided configuration: FactorArguments(strategy='ekfac', use_empirical_fisher=False, amp_dtype=torch.float32, amp_scale=65536.0, has_shared_parameters=False, covariance_max_examples=100000, covariance_data_partitions=1, covariance_module_partitions=1, activation_covariance_dtype=torch.float32, gradient_covariance_dtype=torch.float32, eigendecomposition_dtype=torch.float64, lambda_max_examples=100000, lambda_data_partitions=1, lambda_module_partitions=1, use_iterative_lambda_aggregation=True, offload_activations_to_cpu=True, per_sample_gradient_dtype=torch.float32, lambda_dtype=torch.float32).
INFO:kronfluence.computer.computer:Saved arguments at `/home/ubuntu/subliminal-learning-paraphrasing/influence_results/alpaca_influence/factors_alpaca_factors/factor_arguments.json`.
INFO:kronfluence.computer.computer:DataLoader arguments not provided. Using the configuration: DataLoaderKwargs(num_workers=2, collate_fn=<function default_data_collator a

Computing EKFAC factors...
This may take a while depending on your hardware...


/home/ubuntu/subliminal-learning-paraphrasing/kronfluence/kronfluence/factor/covariance.py:200: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(init_scale=factor_args.amp_scale, enabled=enable_grad_scaler)
Fitting covariance matrices [0/500]   0%|           [time left: ?, time spent: 00:00]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variabl

In [ ]:
# Compute pairwise influence scores
print("Computing influence scores...")
print("This will compute the influence of each training example on the query examples...")

# Configure score computation arguments
score_args = extreme_reduce_memory_score_arguments(
    damping_factor=1e-3,
    dtype=MODEL_DTYPE,
    query_gradient_low_rank=32  # Use low-rank approximation for efficiency
)
score_args.query_gradient_accumulation_steps = 5
score_args.use_full_svd = True

# Compute the influence scores
analyzer.compute_pairwise_scores(
    scores_name="alpaca_scores",
    score_args=score_args,
    factors_name="alpaca_factors",
    query_dataset=query_dataset,
    train_dataset=train_dataset,
    per_device_query_batch_size=1,
    per_device_train_batch_size=2,  # Adjust based on your memory
    overwrite_output_dir=True,
)

print("Influence scores computed successfully!")

In [ ]:
# Load and analyze the computed influence scores
scores = analyzer.load_pairwise_scores("alpaca_scores")
influence_scores = scores["all_modules"]

print(f"Influence scores shape: {influence_scores.shape}")
print(f"Number of query examples: {influence_scores.shape[0]}")
print(f"Number of training examples: {influence_scores.shape[1]}")

# Show statistics
print(f"\nInfluence score statistics:")
print(f"Mean: {influence_scores.mean():.6f}")
print(f"Std: {influence_scores.std():.6f}")
print(f"Min: {influence_scores.min():.6f}")
print(f"Max: {influence_scores.max():.6f}")

In [ ]:
# Find the most influential training examples for each query
def analyze_most_influential(influence_scores, train_dataset, query_dataset, top_k=5):
    """
    Find and display the most influential training examples for each query.
    """
    
    query_examples = [
        "Machine learning explanation",
        "Capital of France", 
        "Python function to add numbers"
    ]
    
    results = []
    
    for query_idx in range(influence_scores.shape[0]):
        query_scores = influence_scores[query_idx]
        
        # Get top influential examples (highest positive influence)
        top_indices = torch.topk(query_scores, top_k).indices
        top_scores = torch.topk(query_scores, top_k).values
        
        print(f"\n" + "="*80)
        print(f"QUERY {query_idx + 1}: {query_examples[query_idx]}")
        print("="*80)
        
        query_result = {
            'query_idx': query_idx,
            'query_description': query_examples[query_idx],
            'top_influential': []
        }
        
        for rank, (idx, score) in enumerate(zip(top_indices, top_scores)):
            idx = idx.item()
            score = score.item()
            
            # Get the original training example
            train_example = train_data[idx]
            
            print(f"\nRank {rank + 1}: Score = {score:.2f}")
            print(f"Training Example {idx}:")
            print(f"Prompt: {train_example['prompt'][:200]}{'...' if len(train_example['prompt']) > 200 else ''}")
            print(f"Completion: {train_example['completion'][:200]}{'...' if len(train_example['completion']) > 200 else ''}")
            print(f"Teacher: {train_example['teacher']}")
            print("-" * 40)
            
            query_result['top_influential'].append({
                'rank': rank + 1,
                'train_idx': idx,
                'score': score,
                'prompt': train_example['prompt'],
                'completion': train_example['completion'],
                'teacher': train_example['teacher']
            })
        
        results.append(query_result)
    
    return results

# Analyze the most influential examples
print("Finding most influential training examples...")
influence_analysis = analyze_most_influential(influence_scores, train_dataset, query_dataset, top_k=10)

In [ ]:
# Save results to JSON for further analysis
output_file = "/home/ubuntu/subliminal-learning-paraphrasing/influence_results.json"

# Convert tensors to regular Python types for JSON serialization
json_results = []
for result in influence_analysis:
    json_result = {
        'query_idx': result['query_idx'],
        'query_description': result['query_description'],
        'top_influential': [
            {
                'rank': item['rank'],
                'train_idx': item['train_idx'],
                'score': float(item['score']),
                'prompt': item['prompt'],
                'completion': item['completion'],
                'teacher': item['teacher']
            }
            for item in result['top_influential']
        ]
    }
    json_results.append(json_result)

# Save to file
with open(output_file, 'w') as f:
    json.dump(json_results, f, indent=2)

print(f"\nResults saved to: {output_file}")
print("\nAnalysis complete!")
print("\nSummary:")
print(f"- Analyzed {len(query_dataset)} query examples")
print(f"- Against {len(train_dataset)} training examples")
print(f"- Found top 10 most influential training examples for each query")
print(f"- Results saved to JSON file for further analysis")

In [ ]:
# Additional analysis: Look at overall patterns
print("\n" + "="*80)
print("OVERALL INFLUENCE PATTERNS")
print("="*80)

# Find the training examples that are most influential overall (across all queries)
overall_influence = influence_scores.sum(dim=0)  # Sum influence across all queries
top_overall_indices = torch.topk(overall_influence, 10).indices
top_overall_scores = torch.topk(overall_influence, 10).values

print("\nTop 10 Most Influential Training Examples (Overall):")
print("-" * 60)

for rank, (idx, score) in enumerate(zip(top_overall_indices, top_overall_scores)):
    idx = idx.item()
    score = score.item()
    
    train_example = train_data[idx]
    
    print(f"\nRank {rank + 1}: Overall Score = {score:.2f}")
    print(f"Training Example {idx}:")
    print(f"Prompt: {train_example['prompt'][:150]}{'...' if len(train_example['prompt']) > 150 else ''}")
    print(f"Completion: {train_example['completion'][:100]}{'...' if len(train_example['completion']) > 100 else ''}")
    print(f"Teacher: {train_example['teacher']}")
    print("-" * 40)

print("\nInfluence analysis complete!")